In [1]:
import Auto_paint_self
slp,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\ERA5-6hour\Mean-sea-level-pressure.nc','msl','yes','time','1980-01-01-00','2014-12-31-18','yes','longitude','yes','latitude',-5.0,53.0,93.0,187.0,0.25,0.25,'no','no',None,None,changeresolution=2,timespace=1,ifchange_west_east='no',ifinterpolate='no')

In [2]:
import numpy as np
slp_HR=np.zeros((slp.shape[0]-4,slp.shape[1]-1,slp.shape[2]-1,5))
slp_HR[:,:,:,0]=slp[:-4,:-1,:-1]
slp_HR[:,:,:,1]=slp[1:-3,:-1,:-1]
slp_HR[:,:,:,2]=slp[2:-2,:-1,:-1]
slp_HR[:,:,:,3]=slp[3:-1,:-1,:-1]
slp_HR[:,:,:,4]=slp[4:,:-1,:-1]
slp_LR=np.zeros((slp.shape[0]-4,int((slp.shape[1]-1)/2),int((slp.shape[2]-1)/2),2))
slp_LR[:,:,:,0]=slp[:-4,:-1:2,:-1:2]
slp_LR[:,:,:,1]=slp[4:,:-1:2,:-1:2]
print(slp_HR.shape,slp_LR.shape)

(51132, 116, 188, 5) (51132, 58, 94, 2)


In [3]:
import numpy as np
slp_HR=(slp_HR-np.nanmean(slp_HR,axis=0))/np.nanstd(slp_HR,axis=0)
slp_LR=(slp_LR-np.nanmean(slp_LR,axis=0))/np.nanstd(slp_LR,axis=0)

In [4]:
slp_HR=np.array(slp_HR)
slp_LR=np.array(slp_LR)

In [5]:
index=int((1-0.2)*slp_HR.shape[0])
trainy=slp_HR[:index,:,:,:]
testy=slp_HR[index:,:,:,:]
trainx=slp_LR[:index,:,:,:]
testx=slp_LR[index:,:,:,:]

In [6]:
print(testx.shape)

(10227, 58, 94, 2)


In [7]:
time_LR=np.array(times[index:-4])
#levels=np.array([0,1,2,3,4])
#lat_HR=np.array(lat[:-1])
#lon_HR=np.array(lon[:-1])
lat_LR=np.array(lat[:-1:2])
lon_LR=np.array(lon[:-1:2])

In [9]:
#import xarray as xr
#testx_mid=np.zeros((testy.shape[0],testy.shape[1],testy.shape[2],2))
#testx1=xr.DataArray(np.array(testx[:,:,:,0]), [('time',time_LR),('latitude',lat_LR),('longitude',lon_LR)])
#testx2=xr.DataArray(np.array(testx[:,:,:,1]), [('time',time_LR),('latitude',lat_LR),('longitude',lon_LR)])
testx_mid[:,:,:,0]=testx1.interp(latitude=np.arange(53.0,-5.0,-0.5),longitude=np.arange(93.0,187.0,0.5),method='linear')
testx_mid[:,:,:,1]=testx2.interp(latitude=np.arange(53.0,-5.0,-0.5),longitude=np.arange(93.0,187.0,0.5),method='linear')

In [10]:
print(testx_mid.shape)

(10227, 116, 188, 2)


In [11]:
def Auto_optical_flow(vx,vy=None):
    import cv2
    import numpy as np
    from scipy.stats import pearsonr

    middle_frame=np.zeros((vx.shape[0],vx.shape[1],vx.shape[2]))
    for i in range(vx.shape[0]):
        flow = cv2.calcOpticalFlowFarneback(vx[i,:,:,0], vx[i,:,:,1], None, 0.5, 3, 15, 3, 5, 1.1, 0)
        flow_horizontal = flow[..., 0]
        flow_vertical = flow[..., 1]
        height, width = vx[i,:,:,:].shape[:2]
        grid_x, grid_y = np.meshgrid(np.arange(width), np.arange(height))
        half_flow_horizontal = grid_x + flow_horizontal * 0.5
        half_flow_vertical = grid_y + flow_vertical * 0.5
        middle_frame[i,:,:] = cv2.remap(vx[i,:,:,0], half_flow_horizontal.astype(np.float32), half_flow_vertical.astype(np.float32), interpolation=cv2.INTER_LINEAR)  
    try:
        r=np.zeros((vy.shape[1],vy.shape[2]))
        p=np.zeros((vy.shape[1],vy.shape[2]))
        for i in range(vy.shape[1]):
            for j in range(vy.shape[2]):
                r[i,j],p[i,j] = pearsonr(middle_frame[:,i,j],vy[:,i,j])
                r=np.nan_to_num(r,nan=0)
    except:
        r=0
        p=0
        
    return middle_frame,vy,r,p

In [12]:
middle_frame_mid,vy,r,p=Auto_optical_flow(testx_mid,vy=None)

In [13]:
print(middle_frame_mid.shape)

(10227, 116, 188)


In [14]:
testx_s_d_0=np.zeros((testy.shape[0],testy.shape[1],testy.shape[2],2))
testx_s_d_1=np.zeros((testy.shape[0],testy.shape[1],testy.shape[2],2))
testx_s_d_0[:,:,:,0]=testx_mid[:,:,:,0]
testx_s_d_0[:,:,:,1]=middle_frame_mid
testx_s_d_1[:,:,:,0]=middle_frame_mid
testx_s_d_1[:,:,:,1]=testx_mid[:,:,:,1]

In [15]:
predicty=np.zeros((testy.shape[0],testy.shape[1],testy.shape[2],5))
predicty[:,:,:,0]=testx_mid[:,:,:,0]
predicty[:,:,:,1],vy,r,p=Auto_optical_flow(testx_s_d_0,vy=None)
predicty[:,:,:,2]=middle_frame_mid
predicty[:,:,:,3],vy,r,p=Auto_optical_flow(testx_s_d_1,vy=None)
predicty[:,:,:,4]=testx_mid[:,:,:,1]

In [16]:
mse_mid=np.zeros((testy.shape[1]-2,testy.shape[2]-2,3))
pearson_mid=np.zeros((testy.shape[1]-2,testy.shape[2]-2,3))
psnr_mid=np.zeros((3))
ssim_mid=np.zeros((3))
mse=np.zeros((testy.shape[1]-2,testy.shape[2]-2,5))
pearson=np.zeros((testy.shape[1]-2,testy.shape[2]-2,5))
psnr=np.zeros((5))
ssim=np.zeros((5))

In [17]:
print(np.isnan(predicty).any())
print(np.sum(np.isnan(predicty)))
print(predicty.shape[0]*predicty.shape[1]*predicty.shape[2]*predicty.shape[3])
print(np.sum(np.isnan(predicty))/(predicty.shape[0]*predicty.shape[1]*predicty.shape[2]*predicty.shape[3]))

True
6197562
1115152080
0.005557593543653705


In [18]:
from tqdm import tqdm
from scipy.stats import pearsonr
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
for k in tqdm(range(5)):
    psnr[k]=tf.image.psnr(np.array(predicty,dtype='float32')[:,1:-1,1:-1,k],np.array(testy,dtype='float32')[:,1:-1,1:-1,k],max_val=(np.nanmax(np.array(slp_HR[:,1:-1,1:-1,k],dtype='float32'))-np.nanmin(np.array(slp_HR[:,1:-1,1:-1,k],dtype='float32'))))
    ssim[k]=tf.image.ssim(np.array(predicty,dtype='float32')[:,1:-1,1:-1,k],np.array(testy,dtype='float32')[:,1:-1,1:-1,k],max_val=(np.nanmax(np.array(slp_HR[:,1:-1,1:-1,k],dtype='float32'))-np.nanmin(np.array(slp_HR[:,1:-1,1:-1,k],dtype='float32'))))
    for i in range(testy.shape[1]-2):
        for j in range(testy.shape[2]-2):
            mse[i,j,k]=mean_squared_error(testy[:,i+1,j+1,k],predicty[:,i+1,j+1,k])
            pearson[i,j,k],_=pearsonr(testy[:,i+1,j+1,k],predicty[:,i+1,j+1,k])
print(np.nanmean(mse))
print(np.nanmean(pearson))
print(np.nanmean(psnr))
print(np.nanmean(ssim))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [07:26<00:00, 89.33s/it]

0.47490103036113374
0.9756593028721641
43.84266738891601
0.6732125878334045


In [19]:
from tqdm import tqdm
for k in tqdm(range(3)):
    psnr_mid[k]=tf.image.psnr(np.array(predicty,dtype='float32')[:,1:-1,1:-1,k+1],np.array(testy,dtype='float32')[:,1:-1,1:-1,k+1],max_val=(np.nanmax(np.array(slp_HR[:,1:-1,1:-1,k+1],dtype='float32'))-np.nanmin(np.array(slp_HR[:,1:-1,1:-1,k+1],dtype='float32'))))
    ssim_mid[k]=tf.image.ssim(np.array(predicty,dtype='float32')[:,1:-1,1:-1,k+1],np.array(testy,dtype='float32')[:,1:-1,1:-1,k+1],max_val=(np.nanmax(np.array(slp_HR[:,1:-1,1:-1,k+1],dtype='float32'))-np.nanmin(np.array(slp_HR[:,1:-1,1:-1,k+1],dtype='float32'))))
    for i in range(testy.shape[1]-2):
        for j in range(testy.shape[2]-2):
            mse_mid[i,j,k]=mean_squared_error(testy[:,i+1,j+1,k+1],predicty[:,i+1,j+1,k+1])
            pearson_mid[i,j,k],_=pearsonr(testy[:,i+1,j+1,k+1],predicty[:,i+1,j+1,k+1])
print(np.nanmean(mse_mid))
print(np.nanmean(pearson_mid))
print(np.nanmean(psnr_mid))
print(np.nanmean(ssim_mid))

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:42<00:00, 34.25s/it]

0.7906455840450801
0.8946731410946014
32.686150868733726
0.45595161120096844


In [20]:
#创建nc文件
def create_nc(times,lat,lon,levels,v,timename,latname,lonname,v_name,iflevel,levelname,iftime,iflat,iflon,newfilename):
    import numpy as np
    from netCDF4 import Dataset,num2date,date2num
    from datetime import datetime
    import pandas as pd
    v = np.array(v)
    if iflat!='no':
        lat = np.array(lat)
    if iflon!='no':
        lon = np.array(lon)
    if iflevel!='no' and iflevel!='yes':
        levels = np.array(levels)
    if iftime !='no':
        times = np.array(times)
    if iftime !='no' and iftime!='self':
        time0=pd.to_datetime(str(np.array(times[0]))).strftime('%Y-%m-%d %H:%M:%S') #1981-01-01 00:00:00
    newnc = Dataset(newfilename,'w',encoding='gbk',format='NETCDF4')
    if iftime !='no':
        newnc.createDimension(timename,None)
        if iftime !='self':
            time = newnc.createVariable(timename,np.float64,(timename,))
        else:
            time = newnc.createVariable(timename,np.int32,(timename,))
        if iflat !='no':
            newnc.createDimension(latname,len(lat))
            lats = newnc.createVariable(latname,np.float32,(latname,))
            if iflon !='no':
                newnc.createDimension(lonname,len(lon))
                lons = newnc.createVariable(lonname,np.float32,(lonname,))
                if iflevel != 'no':
                    newnc.createDimension(levelname,len(levels))
                    levelss = newnc.createVariable(levelname,np.int32,(levelname,))
                    vs = newnc.createVariable(v_name,np.float32,(timename,levelname,latname,lonname,))
                else:
                    vs = newnc.createVariable(v_name,np.float32,(timename,latname,lonname,))
            else:
                if iflevel != 'no':
                    newnc.createDimension(levelname,len(levels))
                    levelss = newnc.createVariable(levelname,np.int32,(levelname,))
                    vs = newnc.createVariable(v_name,np.float32,(timename,levelname,latname,))
                else:
                    vs = newnc.createVariable(v_name,np.float32,(timename,latname,))
        else:
            if iflon !='no':
                newnc.createDimension(lonname,len(lon))
                lons = newnc.createVariable(lonname,np.float32,(lonname,))
                if iflevel != 'no':
                    newnc.createDimension(levelname,len(levels))
                    levelss = newnc.createVariable(levelname,np.int32,(levelname,))
                    vs = newnc.createVariable(v_name,np.float32,(timename,levelname,lonname,))
                else:
                    vs = newnc.createVariable(v_name,np.float32,(timename,lonname,))
            else:
                if iflevel != 'no':
                    newnc.createDimension(levelname,len(levels))
                    levelss = newnc.createVariable(levelname,np.int32,(levelname,))
                    vs = newnc.createVariable(v_name,np.float32,(timename,levelname,))
                else:
                    vs = newnc.createVariable(v_name,np.float32,(timename,))
    else:
        if iflat !='no':
            newnc.createDimension(latname,len(lat))
            lats = newnc.createVariable(latname,np.float32,(latname,))
            if iflon !='no':
                newnc.createDimension(lonname,len(lon))
                lons = newnc.createVariable(lonname,np.float32,(lonname,))
                if iflevel != 'no':
                    newnc.createDimension(levelname,len(levels))
                    levelss = newnc.createVariable(levelname,np.int32,(levelname,))
                    vs = newnc.createVariable(v_name,np.float32,(levelname,latname,lonname,))
                else:
                    vs = newnc.createVariable(v_name,np.float32,(latname,lonname,))
            else:
                if iflevel != 'no':
                    newnc.createDimension(levelname,len(levels))
                    levelss = newnc.createVariable(levelname,np.int32,(levelname,))
                    vs = newnc.createVariable(v_name,np.float32,(levelname,latname,))
                else:
                    vs = newnc.createVariable(v_name,np.float32,(latname,))
        else:
            if iflon !='no':
                newnc.createDimension(lonname,len(lon))
                lons = newnc.createVariable(lonname,np.float32,(lonname,))
                if iflevel != 'no':
                    newnc.createDimension(levelname,len(levels))
                    levelss = newnc.createVariable(levelname,np.int32,(levelname,))
                    vs = newnc.createVariable(v_name,np.float32,(levelname,lonname,))
                else:
                    vs = newnc.createVariable(v_name,np.float32,(lonname,))
            else:
                if iflevel != 'no':
                    newnc.createDimension(levelname,len(levels))
                    levelss = newnc.createVariable(levelname,np.int32,(levelname,))
                    vs = newnc.createVariable(v_name,np.float32,(levelname,))
                else:
                    vs = newnc.createVariable(v_name,np.float32,())
    if iftime !='no':
        newnc.variables[timename][:] = times
        if iftime !='self':
            time.units = 'day since '+str(time0)
            time.calendar = 'gregorian'
            dates = []
            for n in range(v.shape[0]):
                dates.append(datetime(int(pd.to_datetime(str(np.array(times[n]))).strftime('%Y')),int(pd.to_datetime(str(np.array(times[n]))).strftime('%m')),int(pd.to_datetime(str(np.array(times[n]))).strftime('%d')),int(pd.to_datetime(str(times[n])).strftime('%H')),int(pd.to_datetime(str(times[n])).strftime('%M')),int(pd.to_datetime(str(times[n])).strftime('%S'))))
            time[:] = date2num(dates,units = time.units,calendar = time.calendar)
            dates = num2date(time[:],units=time.units,calendar = time.calendar)
    if iflat !='no':
        newnc.variables[latname][:] = lat
    if iflon !='no':
        newnc.variables[lonname][:] = lon
    if iflevel !='no':
        newnc.variables[levelname][:] = levels
    newnc.variables[v_name][:] = np.array(v)
    
    newnc.close()

In [21]:
time=np.array(times[index:-4])
levels=np.array([0,1,2,3,4])
lat=np.array(lat[:-1])
lon=np.array(lon[:-1])
create_nc(time,lat,lon,levels,predicty.transpose(0,3,1,2),'time','latitude','longitude','predicty','all','level','yes','yes','yes',r'E:\Dr_Research\result\ST_model.nc')